In [8]:
import os
import pandas as pd
import panel as pn
import altair as alt
from toolz.curried import pipe

def json_dir(data, data_dir='altairdata'):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=data_dir + '/{prefix}-{hash}.{extension}') )


alt.renderers.enable('default')
pn.extension('vega')

alt.data_transformers.register('json_dir', json_dir)
alt.data_transformers.enable('json_dir', data_dir='mydata')


DataTransformerRegistry.enable('json_dir')

In [9]:
# Lendo os dados do arquivo XLSX
file_path = '../../scrapper-dataset/excel/all-companies.xlsx'

df_reclamacoes = pd.read_excel(file_path)

In [10]:
# Lendo os dados do arquivo XLSX
file_path = '../../scrapper-dataset/censo2022.xlsx'

df_populacao = pd.read_excel(file_path)

In [11]:
df_populacao.head()

,estado,Total
0,RO,729955
1,AC,319315
2,AM,1307114
3,RR,211862
4,PA,3056893


In [12]:
df_quantidade_reclamacoes = df_reclamacoes.groupby('estado').size().reset_index(name='Quantidade_Reclamacoes')
df_merge = df_quantidade_reclamacoes.merge(df_populacao, on='estado', how='left')
df_merge['Percent_Relativo'] = df_merge['Quantidade_Reclamacoes'] / df_merge['Total'] * 100
df_merge_sorted = df_merge.sort_values('Percent_Relativo', ascending=False)
# df_merge_sorted


In [13]:
df_merge.head()

,estado,Quantidade_Reclamacoes,Total,Percent_Relativo
0,********,2,NaN,NaN
1,-,4,NaN,NaN
2,--,140,NaN,NaN
3,AC,178,319315.0,0.055744
4,AL,986,1322185.0,0.074574


In [14]:
bars = alt.Chart(df_merge_sorted).mark_bar().encode(
  y=alt.Y('estado:N', sort='-x'),
  x='Percent_Relativo:Q',
  color='estado:N',
  tooltip=['estado', 'Percent_Relativo:Q']

)

text = bars.mark_text(
    align='center',
    baseline='middle',
    dx=+25,
).encode(
    text=alt.Text('Percent_Relativo:Q', format=".4f")
)

chart = (bars + text).properties(
    title='% de reclamações por população',
    width=400,
    height=1000
)

bars_all = alt.Chart(df_reclamacoes).mark_bar().encode(
    x='count()',
    y=alt.Y('estado:N', sort='-x'),
    color='estado:N',
    tooltip=['estado', 'count()']

)

text_all = bars_all.mark_text(
    align='center',
    baseline='middle',
    # dy=+5,
    dx=+25,
).encode(
    text=alt.Text('count()', format=".0f")
)

chart_all = (bars_all + text_all).properties(
    title='Quantidade de Reclamações Totais por Estado ',
    width=400,
    height=1000
)

# combined_chart = alt.layer(chart_all, chart).resolve_scale(y='independent')

# combined_chart
chart_all | chart  

alt.HConcatChart(...)